In [2]:
import random
hello_dict = { 0 : "hello!", 1 : "I'm here", 2 : "hey!" ,3 : "what's up?" }
Goodbye_dict = { 0 : "Goodbye!", 1 : "Nice to see you", 2 : "ok!" ,3 : "Well.." }
num = random.randrange(4)
hello_dict[num]

'hey!'

In [35]:
import json
a = "www"
c = "qqqq"
# Opening JSON file
with open(r'..\docs\user.json') as f:
    data = json.load(f)
"U39ef08913a1aa5a8e5f100399fe9fd2c"in data
# data[a] = c
# print(data)
# with open(r'D:\linebot_self\docs\user.json', 'w') as f:
#     json.dump(data, f,ensure_ascii=False)


True

In [38]:
import requests
from datetime import datetime,timedelta
def weather(WEATHER_API_KEY,user):

    now = datetime.now().date()
    weather_apikey = WEATHER_API_KEY
    weather_apikey = 'CWB-B64CD8B7-02BF-441E-B253-C654F478E513'
    dataid = 'F-D0047-069' #新北市
    element_name = "T,Wx,PoP6h,AT"

    url = f'https://opendata.cwb.gov.tw/api/v1/rest/datastore/{dataid}?Authorization={weather_apikey}&locationName={user[2:5]}&elementName={element_name}'
    response = requests.get(url)
    data = response.json()

    date_code = {'今天':now,'明天':now + timedelta(days=1)}
    
    extra = data['records']['locations'][0]['location'][0]['weatherElement']

    relpy_text = user[2:] +"\n"

    for j in range(len(extra[0]['time'])):
        # 讀取資料第一筆時間
        moment = datetime.strptime(extra[0]['time'][j]['startTime'], '%Y-%m-%d %H:%M:%S')
        
        # 判斷所需的日期天氣資料

        if str(date_code[user[:2]]) in str(moment):

            relpy_text+= str(moment) 
            relpy_text+='\n'
            for i in range(len(extra)-1):
                description = extra[i]['description']
                try:
                    wx = extra[i]['time'][j]['elementValue'][0]['value']
                except IndexError:
                    wx = '尚未預測'
                relpy_text+=description+":"+ wx 
                relpy_text+='\n'

            try:
                relpy_text+=extra[-1]['description']+":"+extra[-1]['time'][j//2]['elementValue'][0]['value']+"%"
                relpy_text+='\n'
                relpy_text+='\n'
            except IndexError:
                relpy_text+='N\A'
                
    return relpy_text
print(weather(1,"明天板橋區天氣"))

板橋區天氣
2023-03-20 00:00:00
天氣現象:多雲
體感溫度:19
溫度:19
6小時降雨機率:20%

2023-03-20 03:00:00
天氣現象:多雲
體感溫度:20
溫度:19
6小時降雨機率:20%

2023-03-20 06:00:00
天氣現象:多雲
體感溫度:20
溫度:18
6小時降雨機率:0%

2023-03-20 09:00:00
天氣現象:多雲
體感溫度:23
溫度:21
6小時降雨機率:0%

2023-03-20 12:00:00
天氣現象:多雲
體感溫度:28
溫度:26
6小時降雨機率:30%

2023-03-20 15:00:00
天氣現象:短暫雨
體感溫度:27
溫度:26
6小時降雨機率:30%

2023-03-20 18:00:00
天氣現象:晴
體感溫度:25
溫度:23
6小時降雨機率:0%

2023-03-20 21:00:00
天氣現象:晴
體感溫度:24
溫度:22
6小時降雨機率:0%




In [36]:
now = datetime.now().date()
now

datetime.date(2023, 3, 19)